In [9]:
import sys
sys.path.append("../")
import pandas as pd
import openpyxl as oxl

from utils.prompts import render
from utils.llm_client import LLMClient
from utils.logging_utils import log_llm_call
from utils.router import pick_model,should_use_reasoning_model
from IPython.display import display, Markdown

### Load data from the text file

In [10]:
with open('../data/raw/Scenarios.txt') as f:
    text = f.read()
    sample_scenarios = [s.strip() for s in text.split("\n\n") if s.strip()]
    
    print(f"Loaded {len(sample_scenarios)} sample scenarios.")

Loaded 2 sample scenarios.


### Instruction of the prompt 

In [11]:
instruction = """
You are an emergency decision-support system.

TASK:
Analyze the scenario and respond using EXACTLY this format:

Reasoning:
1. Immediate life threat: <list the most urgent life-threatening issue>
2. Immediate health threat: <list critical medical issues>
3. Other risk: <list other risks, e.g., logistics, hunger, environment>
4. Prioritized actions:
   - For each issue above, list ONE concise action per bullet.
   - Each action MUST be one short sentence.
   - Do NOT combine multiple tasks in a single action.

Answer:
1. Immediate Life Threat: <combine first issue with its action>
2. Immediate Health Threat: <combine second issue with its action>
3. Other Risk: <combine third issue with its action>

RULES:
- Each action in the Prioritized Actions section must be **one short sentence**.
- Do NOT combine multiple actions in a single bullet.
- Keep bullets **concise and factual**.
- No paragraphs, introductions, reasoning explanations, or creative text.
- Maintain this structure exactly for every scenario.
"""



### LLM Client

In [12]:
model = pick_model('google', 'cot')
client = LLMClient('google', model)

prompt_text, spec = render(
    'cot_reasoning.v1',
    role='Emergency Decision Supporter',
    problem=sample_scenarios
)

### Get the Output

In [ ]:
import time

for scenario in sample_scenarios:
    full_prompt = f"{prompt_text}\n\n{instruction}\n\nScenario:\n{scenario}"


    display(Markdown(f"{scenario}\n"))

    # run Choas mode 3 times
    for run in range(1, 4): 
        messages = [{'role': 'user', 'content': full_prompt}]
        response = client.chat(messages, temperature=1.0, max_tokens=spec.max_tokens)

        display(Markdown(f"### Chaos Run {run}"))
        display(Markdown(response['text']))
        time.sleep(2)


    # run Safe mode once
    messages = [{'role': 'user', 'content': full_prompt}]
    response_safe = client.chat(messages, temperature=0.0, max_tokens=spec.max_tokens)

    display(Markdown("### Safe Run"))
    display(Markdown(response_safe['text']))
    time.sleep(2)

    # Optional separator between scenarios
    display(Markdown("---"))


SCENARIO A: THE KANDY LANDSLIDE
Location: Hanthana Tea Factory
Details: "We are trapped. The access road is blocked. My uncle is stuck in the line rooms below, climbing a tree as water rises (Immediate Life Threat). However, inside the factory, we have a diabetic patient who has collapsed and needs insulin (Immediate Health Threat). We also have 40 people hungry."


### Chaos Run 1

Reasoning:
1. Immediate life threat: Uncle trapped in rising water while climbing a tree in the line rooms.
2. Immediate health threat: Diabetic patient collapsed inside the factory and requiring insulin.
3. Other risk: Forty people hungry and isolation due to a blocked access road.
4. Prioritized actions:
   - Dispatch a water rescue team to the uncle's location.
   - Administer insulin to the collapsed diabetic patient.
   - Distribute available food supplies to the hungry group.

Answer:
1. Immediate Life Threat: Uncle trapped in rising water requires immediate rescue team dispatch.
2. Immediate Health Threat: Collapsed diabetic patient requires urgent insulin administration.
3. Other Risk: Forty hungry people require immediate food distribution.

### Chaos Run 2

Reasoning:
1. Immediate life threat: Person trapped in a tree with rising water levels.
2. Immediate health threat: Collapsed diabetic patient requiring insulin.
3. Other risk: Forty people without food supplies.
4. Prioritized actions:
   - Dispatch water rescue personnel to evacuate the person from the tree.
   - Deliver insulin and medical assistance to the collapsed patient.
   - Coordinate the delivery of food rations to the factory.

Answer:
1. Immediate Life Threat: Person trapped in tree by rising water requires immediate rescue team dispatch.
2. Immediate Health Threat: Collapsed diabetic patient needs urgent delivery of insulin and medical stabilization.
3. Other Risk: Forty hungry people require the coordination and delivery of emergency food supplies.

...

### Chaos Run 3

Reasoning:
1. Immediate life threat: ICU ventilator failure within two hours due to power loss.
2. Immediate health threat: Flood water entering the ward housing 50 bedridden elderly patients.
3. Other risk: Thirty-minute delay and capacity limitations of the incoming generator truck.
4. Prioritized actions:
   - Direct the incoming generator truck to power the ICU.
   - Move all bedridden patients to higher floors.
   - Prepare manual ventilation equipment for immediate use.

Answer:
1. Immediate Life Threat: ICU ventilator failure - Direct the incoming generator truck to power the ICU.
2. Immediate Health Threat: Flooded ground floor ward - Move all bedridden patients to higher floors.
3. Other Risk: Generator arrival delay - Prepare manual ventilation equipment for immediate use.

### Safe Run

Reasoning:
1. Immediate life threat: 3 patients on ventilators with 2-hour battery life.
2. Immediate health threat: 50 bedridden elderly patients in a flooding ward.
3. Other risk: Limited generator capacity.
4. Prioritized actions:
   - Connect the generator truck to the ICU.
   - Move the 50 elderly patients to higher floors.
   - Request additional generators from nearby facilities.

Answer:
1. Immediate Life Threat: 3 patients on ventilators with 2-hour battery require the generator truck to be connected to the ICU.
2. Immediate Health Threat: 50 bedridden elderly patients in the flooding ward must be moved to higher floors.
3. Other Risk: Limited generator capacity requires requesting additional

---

# Deliverables

## 1. Observation on Scneario A

### Safe Mode

- Life Threats: Correctly identifies uncle trapped in tree.

- Health Threats: Diabetic patient correctly listed.

- Other Risks: 40 people hungry identified.

- Prioritized Actions: Short, clear, actionable steps (rescue, insulin, food).

- Determinism: Fully consistent, repeatable output.

### Chaos Mode

- Life Threats: Mostly correct, but wording varies (“person in tree” instead of “uncle”). Safe Mode kept consistent terminology.

- Health Threats: Correct but phrasing varies; Safe Mode consistently mentions the collapsed diabetic patient.

- Other Risks: Sometimes combined with actions or extra details; Safe Mode keeps them separate.

- Prioritized Actions: Introduces creative/longer actions, e.g., helicopter or drone, while Safe Mode only uses direct resources like water rescue or insulin delivery.

- Determinism: Output varies across runs, unlike the stable Safe Mode output.

### Drift / Hallucination Examples

- Chaos Run 2: “Deliver insulin via emergency air transport” → Safe Mode only used standard insulin delivery.

- Chaos Run 3: “Dispatch a helicopter for immediate aerial rescue” → Safe Mode used water rescue teams.

- Chaos Run 3: “Drop emergency food rations” → Safe Mode simply coordinated direct food delivery.

## 2. Observation on Scneario B

### Safe Mode

- Life Threats: ICU ventilator patients correctly identified.

- Health Threats: 50 bedridden elderly patients correctly identified.

- Other Risks: Limited generator capacity recognized.

- Prioritized Actions: Clear, concise, realistic steps.

- Determinism: Stable, repeatable output.

### Chaos Mode

- Life Threats: Correct but wording varies (“ICU ventilator failure within 2 hours”), whereas Safe Mode uses exact scenario language (“3 ICU patients on ventilators with 2-hour battery”).

- Health Threats: More verbose phrasing; Safe Mode keeps it concise.

- Other Risks: Sometimes combined with other actions or expanded; Safe Mode separates clearly.

- Prioritized Actions: Adds extra steps/resources (manual ventilation, reassigning generator), while Safe Mode only executes implied actions.

- Determinism: Chaos outputs inconsistent across runs; Safe Mode is fully consistent.

### Drift / Hallucination Examples

- Chaos Run 2: “Assign medical staff to perform manual ventilation if power fails” → Safe Mode did not add this; only connected generator.

- Chaos Run 3: “Prepare manual ventilation equipment for immediate use” → Safe Mode only moved patients and connected generator.

- Chaos Run 3: “Generator arrival delay emphasized differently” → Safe Mode uses a single straightforward prioritization.